In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import json

torch.random.manual_seed(0)

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

task_prompt_path = "prompt_stack_two.txt"
extra_prompt = 'Please generate 1 similiar questions and descriptions with difference wordings, but make sure all the information are there. Do not answer the question'
input_prompt = open(task_prompt_path, 'r').read() + extra_prompt

messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": input_prompt},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.5,
    "do_sample": True,
}

all_output = []
for i in range(2):
    output = pipe(messages, **generation_args)
    all_output.append(output[0]['generated_text'])
    
response_number_per_question = 2
# repeat the same question response_number_per_question times
all_output = [item for item in all_output for i in range(response_number_per_question)]

print (len(all_output))

all_message = []
for j in range(len(all_output)):
    message = [{"role": "user", "content": all_output[j]}]
    output = pipe(message, **generation_args)
    all_message.append(output[0]['generated_text'])

# Combine the questions and answers into a list of dictionaries
qa_pairs = [{"input": question, "output": answer} for question, answer in zip(all_output, all_message)]

# Save the combined data into a JSON file
with open('qa_dataset.json', 'w') as f:
    json.dump(qa_pairs, f, indent=4)

# Print the generated questions and answers
for i in range(len(all_output)):
    print("question:", all_output[i])
    print("answer:", all_message[i])
    print("-------------------")


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4
question:  Step by step plans:

1. Identify the graspable points of cubeA and cubeB.
2. Move the robotic arm to the graspable point of cubeA.
3. Open the gripper.
4. Move the gripper to grasp cubeA.
5. Close the gripper to secure cubeA.
6. Move the robotic arm to the graspable point of cubeB.
7. Open the gripper.
8. Move the gripper to grasp cubeB.
9. Close the gripper to secure cubeB.
10. Move the robotic arm to a position where cubeA is above cubeB.
11. Close the gripper of cubeA.
12. Move the robotic arm to lift cubeA.
13. Lift cubeA to the desired height.
14. Move the robotic arm to a position where cubeA is directly above cubeB.
15. Open the gripper of cubeA.
16. Move the robotic arm to release cubeA onto cubeB.

Python code:

```python
import numpy as np

# Open gripper
open_gripper()

# Get graspable point of cubeA
cubeA_graspable_point = get_graspable_point('cubeA')

# Move to graspable point of cubeA
move_to_position(cubeA_graspable_point)

# Open gripper to grasp cubeA
open

In [26]:
import openai
import os
import time
import argparse

def open_ai_call_with_retry(model_name, messages, temperature=0.0, seed=1000, max_tokens=300):
    reset_trigger_phrases = ["CHOICE", "NUM"]
    success = False
    while not success:
        try:
            # Initialize the OpenAI client
            openai.api_key = os.getenv("OPENAI_API_KEY")
            completion = openai.ChatCompletion.create(
                model=model_name,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                # seed=seed,
            )
            success = True
            for reset_trigger_phrase in reset_trigger_phrases:
                if reset_trigger_phrase in completion.choices[0].message['content']:
                    success = False
        except Exception as e:
            print("OpenAI API call issue, re-trying..." + str(e) + "\n")
            time.sleep(5)
    return completion


model = "gpt-4o"

task_prompt_path = "data_prep_stack_two.txt"
input_prompt = open(task_prompt_path, 'r').read()


messages = [
    {"role": "user", "content": input_prompt},
]


all_output = []
for i in range(3):
    output = open_ai_call_with_retry(model, messages, temperature=0.7, max_tokens=2000).choices[0].message['content']
    all_output.append(output)
    
response_number_per_question = 1
# repeat the same question response_number_per_question times
all_output = [item for item in all_output for i in range(response_number_per_question)]

print (len(all_output))

all_message = []
for j in range(len(all_output)):
    message = [{"role": "user", "content": all_output[j]}]
    output = open_ai_call_with_retry(model, message, temperature=0.4, max_tokens=2000).choices[0].message['content']
    all_message.append(output)

# Combine the questions and answers into a list of dictionaries
qa_pairs = [{"input": question, "output": answer} for question, answer in zip(all_output, all_message)]

# Save the combined data into a JSON file
with open('two_stack_dataset.json', 'w') as f:
    json.dump(qa_pairs, f, indent=4)

# Print the generated questions and answers
# for i in range(len(all_output)):
#     print("question:", all_output[i])
#     print("answer:", all_message[i])
#     print("-------------------")

3


In [ ]:
import openai
import os
import time
import argparse

def open_ai_call_with_retry(model_name, messages, temperature=0.0, seed=1000, max_tokens=300):
    reset_trigger_phrases = ["CHOICE", "NUM"]
    success = False
    while not success:
        try:
            # Initialize the OpenAI client
            openai.api_key = os.getenv("OPENAI_API_KEY")
            completion = openai.ChatCompletion.create(
                model=model_name,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                # seed=seed,
            )
            success = True
            for reset_trigger_phrase in reset_trigger_phrases:
                if reset_trigger_phrase in completion.choices[0].message['content']:
                    success = False
        except Exception as e:
            print("OpenAI API call issue, re-trying..." + str(e) + "\n")
            time.sleep(5)
    return completion


model = "gpt-4o"

task_prompt_path = "data_prep_stack_three.txt"
input_prompt = open(task_prompt_path, 'r').read()


messages = [
    {"role": "user", "content": input_prompt},
]


all_output = []
for i in range(3):
    output = open_ai_call_with_retry(model, messages, temperature=0.7, max_tokens=2000).choices[0].message['content']
    all_output.append(output)
    
response_number_per_question = 1
# repeat the same question response_number_per_question times
all_output = [item for item in all_output for i in range(response_number_per_question)]

print (len(all_output))

all_message = []
for j in range(len(all_output)):
    message = [{"role": "user", "content": all_output[j]}]
    output = open_ai_call_with_retry(model, message, temperature=0.4, max_tokens=2000).choices[0].message['content']
    all_message.append(output)

# Combine the questions and answers into a list of dictionaries
qa_pairs = [{"input": question, "output": answer} for question, answer in zip(all_output, all_message)]

# Save the combined data into a JSON file
with open('three_stack_dataset.json', 'w') as f:
    json.dump(qa_pairs, f, indent=4)

# Print the generated questions and answers
# for i in range(len(all_output)):
#     print("question:", all_output[i])
#     print("answer:", all_message[i])
#     print("-------------------")